In [154]:
obs1 = [-1 -1 -1 -1 -1 -1 -1 -1 +1 +1 +1 +1 +1 +1 +1 +1; 
                -1 -1 -1 -1 +1 +1 +1 +1 -1 -1 -1 -1 +1 +1 +1 +1;
        -1 -1 +1 +1 -1 -1 +1 +1 -1 -1 +1 +1 -1 -1 +1 +1;
    -1 +1 -1 +1 -1 +1 -1 +1 -1 +1 -1 +1 -1 +1 -1 +1]


tau = 0.5

k = 1000000000


1000000000

In [155]:
q = zeros(16)
for j = 1:16
   q[j] = (1 / 16) * (1 + obs1[1, j] * obs1[2, j] * tau)
end

q


16-element Vector{Float64}:
 0.09375
 0.09375
 0.09375
 0.09375
 0.03125
 0.03125
 0.03125
 0.03125
 0.03125
 0.03125
 0.03125
 0.03125
 0.09375
 0.09375
 0.09375
 0.09375

In [263]:
mean = k *q

using LinearAlgebra # this package comes with Julia, no need to install it

meandia = diagm(q)

b = transpose(q)

c = q * b

d = meandia -c 

cov = k * d
# find the Covariance of the distribution

using Distributions
d = MvNormal(mean, cov)
test = rand(d)
# the basic generated data

16-element Vector{Float64}:
 9.375761058718953e7
 9.374876403521897e7
 9.374690037312461e7
 9.375302872039875e7
 3.1245180717181955e7
 3.1254178511845656e7
 3.124837055588501e7
 3.1246670116113413e7
 3.1241920551602434e7
 3.1247896052321594e7
 3.1250819225579415e7
 3.1248168615593202e7
 9.374816167232552e7
 9.37618864870285e7
 9.376102434706515e7
 9.373941943150829e7

In [4]:
import Pkg; Pkg.add("Ipopt")

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


This is Z6 which will used the fixed data for compartion to the mates' results.

In [6]:
import Pkg; Pkg.add("JuMP")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


This is case 1

In [7]:
using Ipopt,JuMP
model = Model(Ipopt.Optimizer)
@variable(model,x[1:5]>=0)
#set each one variable to be larger than 0
@constraint(model,x[1]<=1)
@constraint(model,x[2]<=1)
@constraint(model,x[3]<=sqrt(k))
@constraint(model,x[4]<=sqrt(k))
@constraint(model,x[5]<=1)
@NLconstraint(model,x[1]*x[2]*x[5] == 0.5)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x[1]*x[3]
                -obs1[2,i]*obs1[4,i]*x[2]*x[4]
                -obs1[1,i]*obs1[4,i]*x[1]*x[5]*x[4]
                -obs1[2,i]*obs1[3,i]*x[2]*x[5]*x[3])^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        3
Number of nonzeros in inequality constraint Jacobian.:        5
Number of nonzeros in Lagrangian Hessian.............:       21

Total number of variables............................:        5
                     variables with only lower bounds:        5
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality co

8.215465594007364

In [264]:
function test1(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x3>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x2*x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3
                -obs1[2,i]*obs1[4,i]*x2*x4
                -obs1[1,i]*obs1[4,i]*x1*x5*x4
                -obs1[2,i]*obs1[3,i]*x2*x5*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test1 (generic function with 2 methods)

In [266]:
function test11(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x2>=0)
@variable(model,x3>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x2*x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x3
                -obs1[2,i]*obs1[4,i]*x2*x4
                -obs1[1,i]*obs1[4,i]*x5*x4
                -obs1[2,i]*obs1[3,i]*x2*x5*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test11 (generic function with 2 methods)

In [267]:
function test12(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x3>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3
                -obs1[2,i]*obs1[4,i]*x4
                -obs1[1,i]*obs1[4,i]*x1*x5*x4
                -obs1[2,i]*obs1[3,i]*x5*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test12 (generic function with 2 methods)

In [268]:
function test13(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x2*x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*0
                -obs1[2,i]*obs1[4,i]*x2*x4
                -obs1[1,i]*obs1[4,i]*x1*x5*x4
                -obs1[2,i]*obs1[3,i]*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test13 (generic function with 2 methods)

In [269]:
function test14(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x3>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x2*x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3
                -obs1[2,i]*obs1[4,i]*0
                -obs1[1,i]*obs1[4,i]*0
                -obs1[2,i]*obs1[3,i]*x2*x5*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test14 (generic function with 2 methods)

In [270]:
function test15(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x3>=0)
@variable(model,x4>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3
                -obs1[2,i]*obs1[4,i]*x2*x4
                -obs1[1,i]*obs1[4,i]*x1*x4
                -obs1[2,i]*obs1[3,i]*x2*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test15 (generic function with 2 methods)

In [271]:
function test111(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x2>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x2<=1)
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x2*x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*0
                -obs1[2,i]*obs1[4,i]*x2*x4
                -obs1[1,i]*obs1[4,i]*x5*x4
                -obs1[2,i]*obs1[3,i]*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test111 (generic function with 2 methods)

In [272]:
function test112(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x2>=0)
@variable(model,x3>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x2*x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3
                -obs1[2,i]*obs1[4,i]*0
                -obs1[1,i]*obs1[4,i]*0
                -obs1[2,i]*obs1[3,i]*x2*x5*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test112 (generic function with 2 methods)

In [273]:
function test113(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*0
                -obs1[2,i]*obs1[4,i]*x4
                -obs1[1,i]*obs1[4,i]*x1*x5*x4
                -obs1[2,i]*obs1[3,i]*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test113 (generic function with 2 methods)

In [274]:
function test114(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x3>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3
                -obs1[2,i]*obs1[4,i]*0
                -obs1[1,i]*obs1[4,i]*0
                -obs1[2,i]*obs1[3,i]*x5*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test114 (generic function with 2 methods)

In [275]:
function test115(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x4>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x4<=sqrt(k))
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*0
                -obs1[2,i]*obs1[4,i]*x2*x4
                -obs1[1,i]*obs1[4,i]*x1*x4
                -obs1[2,i]*obs1[3,i]*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test115 (generic function with 2 methods)

In [276]:
function test116(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x3>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3
                -obs1[2,i]*obs1[4,i]*0
                -obs1[1,i]*obs1[4,i]*0
                -obs1[2,i]*obs1[3,i]*x2*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test116 (generic function with 2 methods)

In [277]:
function test117(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x3>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x3
                -obs1[2,i]*obs1[4,i]*x4
                -obs1[1,i]*obs1[4,i]*x5*x4
                -obs1[2,i]*obs1[3,i]*x5*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test117 (generic function with 2 methods)

In [278]:
function test118(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x3>=0)
@variable(model,x4>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@NLconstraint(model,x1 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3
                -obs1[2,i]*obs1[4,i]*x4
                -obs1[1,i]*obs1[4,i]*x1*x4
                -obs1[2,i]*obs1[3,i]*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test118 (generic function with 2 methods)

In [279]:
function test119(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x2>=0)
@variable(model,x3>=0)
@variable(model,x4>=0)
#set each one variable to be larger than 0
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@NLconstraint(model,x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x3
                -obs1[2,i]*obs1[4,i]*x2*x4
                -obs1[1,i]*obs1[4,i]*x4
                -obs1[2,i]*obs1[3,i]*x2*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test119 (generic function with 2 methods)

In [280]:
function test1111(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*0
                -obs1[2,i]*obs1[4,i]*x4
                -obs1[1,i]*obs1[4,i]*x5*x4
                -obs1[2,i]*obs1[3,i]*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test1111 (generic function with 1 method)

In [281]:
function test1112(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x3>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x3<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x3
                -obs1[2,i]*obs1[4,i]*0
                -obs1[1,i]*obs1[4,i]*0
                -obs1[2,i]*obs1[3,i]*x5*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test1112 (generic function with 1 method)

In [346]:
function test1113(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x4>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x4<=sqrt(k))
@NLconstraint(model,x1 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*0
                -obs1[2,i]*obs1[4,i]*x4
                -obs1[1,i]*obs1[4,i]*x1*x4
                -obs1[2,i]*obs1[3,i]*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test1113 (generic function with 1 method)

In [283]:
function test1114(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x3>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x3<=sqrt(k))
@NLconstraint(model,x1 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3
                -obs1[2,i]*obs1[4,i]*0
                -obs1[1,i]*obs1[4,i]*0
                -obs1[2,i]*obs1[3,i]*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test1114 (generic function with 1 method)

In [284]:
function test1115(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x2>=0)
@variable(model,x3>=0)
#set each one variable to be larger than 0
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@NLconstraint(model,x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x3
                -obs1[2,i]*obs1[4,i]*0
                -obs1[1,i]*obs1[4,i]*0
                -obs1[2,i]*obs1[3,i]*x2*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test1115 (generic function with 1 method)

In [285]:
function test1116(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x2>=0)
@variable(model,x4>=0)
#set each one variable to be larger than 0
@constraint(model,x2<=1)
@constraint(model,x4<=sqrt(k))
@NLconstraint(model,x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*0
                -obs1[2,i]*obs1[4,i]*x2*x4
                -obs1[1,i]*obs1[4,i]*x4
                -obs1[2,i]*obs1[3,i]*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test1116 (generic function with 1 method)

In [289]:
function test2a(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x3>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3*x5
                -obs1[2,i]*obs1[4,i]*x2*x4*x5
                -obs1[1,i]*obs1[4,i]*x1*x5*x4
                -obs1[2,i]*obs1[3,i]*x2*x5*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end


test2a (generic function with 2 methods)

In [287]:
function test2a1(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x2>=0)
@variable(model,x3>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x3*x5
                -obs1[2,i]*obs1[4,i]*x2*x4*x5
                -obs1[1,i]*obs1[4,i]*x5*x4
                -obs1[2,i]*obs1[3,i]*x2*x5*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2a1 (generic function with 2 methods)

In [290]:
function test2a2(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x3>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3*x5
                -obs1[2,i]*obs1[4,i]*x4*x5
                -obs1[1,i]*obs1[4,i]*x1*x5*x4
                -obs1[2,i]*obs1[3,i]*x5*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2a2 (generic function with 2 methods)

In [291]:
function test2a3(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*0*x5
                -obs1[2,i]*obs1[4,i]*x2*x4*x5
                -obs1[1,i]*obs1[4,i]*x1*x5*x4
                -obs1[2,i]*obs1[3,i]*x2*x5*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2a3 (generic function with 2 methods)

In [292]:
function test2a4(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x3>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3*x5
                -obs1[2,i]*obs1[4,i]*x2*0*x5
                -obs1[1,i]*obs1[4,i]*x1*x5*0
                -obs1[2,i]*obs1[3,i]*x2*x5*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2a4 (generic function with 2 methods)

In [293]:
function test2a5(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x3>=0)
@variable(model,x4>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3
                -obs1[2,i]*obs1[4,i]*x2*x4
                -obs1[1,i]*obs1[4,i]*x1*x4
                -obs1[2,i]*obs1[3,i]*x2*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2a5 (generic function with 2 methods)

In [294]:
function test2a11(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x2>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x2<=1)
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*0
                -obs1[2,i]*obs1[4,i]*x2*x4*x5
                -obs1[1,i]*obs1[4,i]*x5*x4
                -obs1[2,i]*obs1[3,i]*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2a11 (generic function with 2 methods)

In [295]:
function test2a12(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*0*x5
                -obs1[2,i]*obs1[4,i]*x4*x5
                -obs1[1,i]*obs1[4,i]*x1*x5*x4
                -obs1[2,i]*obs1[3,i]*x2*x5*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2a12 (generic function with 2 methods)

In [296]:
function test2a13(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x2>=0)
@variable(model,x3>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x3*x5
                -obs1[2,i]*obs1[4,i]*x2*0*x5
                -obs1[1,i]*obs1[4,i]*x1*x5*0
                -obs1[2,i]*obs1[3,i]*x2*x5*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2a13 (generic function with 2 methods)

In [297]:
function test2a14(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x3>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3*x5
                -obs1[2,i]*obs1[4,i]*0
                -obs1[1,i]*obs1[4,i]*0
                -obs1[2,i]*obs1[3,i]*x5*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2a14 (generic function with 2 methods)

In [298]:
function test2a15(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x2>=0)
@variable(model,x3>=0)
@variable(model,x4>=0)
#set each one variable to be larger than 0
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@NLconstraint(model,x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x3
                -obs1[2,i]*obs1[4,i]*x2*x4
                -obs1[1,i]*obs1[4,i]*x4
                -obs1[2,i]*obs1[3,i]*x2*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2a15 (generic function with 2 methods)

In [299]:
function test2a16(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x3>=0)
@variable(model,x4>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@NLconstraint(model,x1== e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3
                -obs1[2,i]*obs1[4,i]*x4
                -obs1[1,i]*obs1[4,i]*x1*x4
                -obs1[2,i]*obs1[3,i]*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2a16 (generic function with 2 methods)

In [300]:
function test2a17(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x4>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x4<=sqrt(k))
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*0
                -obs1[2,i]*obs1[4,i]*x2*x4
                -obs1[1,i]*obs1[4,i]*x1*x4
                -obs1[2,i]*obs1[3,i]*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2a17 (generic function with 2 methods)

In [302]:
function test2a18(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x3>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3
                -obs1[2,i]*obs1[4,i]*0
                -obs1[1,i]*obs1[4,i]*0
                -obs1[2,i]*obs1[3,i]*x2*x3)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2a18 (generic function with 2 methods)

In [303]:
function test2b(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x3>=0) 
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3*x5
                -obs1[2,i]*obs1[4,i]*x2*x4*x5
                -obs1[1,i]*obs1[4,i]*x1*x5*x4
                -obs1[2,i]*obs1[3,i]*x2*x5*x3
                -obs1[1,i]*obs1[2,i]*obs1[3,i]*obs1[4,i]*x1*x2*x3*x4)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2b (generic function with 3 methods)

In [304]:
function test2b1(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x2>=0)
@variable(model,x3>=0) 
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x3*x5
                -obs1[2,i]*obs1[4,i]*x2*x4*x5
                -obs1[1,i]*obs1[4,i]*x5*x4
                -obs1[2,i]*obs1[3,i]*x2*x5*x3
                -obs1[1,i]*obs1[2,i]*obs1[3,i]*obs1[4,i]*x2*x3*x4)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2b1 (generic function with 3 methods)

In [305]:
function test2b2(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x3>=0) 
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3*x5
                -obs1[2,i]*obs1[4,i]*x4*x5
                -obs1[1,i]*obs1[4,i]*x1*x5*x4
                -obs1[2,i]*obs1[3,i]*x5*x3
                -obs1[1,i]*obs1[2,i]*obs1[3,i]*obs1[4,i]*x1*x3*x4)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2b2 (generic function with 3 methods)

In [306]:
function test2b3(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*0*x5
                -obs1[2,i]*obs1[4,i]*x2*x4*x5
                -obs1[1,i]*obs1[4,i]*x1*x5*x4
                -obs1[2,i]*obs1[3,i]*x2*x5*0
                -obs1[1,i]*obs1[2,i]*obs1[3,i]*obs1[4,i]*x1*x2*0*x4)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2b3 (generic function with 3 methods)

In [307]:
function test2b4(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x3>=0) 
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3*x5
                -obs1[2,i]*obs1[4,i]*x2*0*x5
                -obs1[1,i]*obs1[4,i]*x1*x5*0
                -obs1[2,i]*obs1[3,i]*x2*x5*x3
                -obs1[1,i]*obs1[2,i]*obs1[3,i]*obs1[4,i]*x1*x2*x3*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2b4 (generic function with 3 methods)

In [308]:
function test2b5(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x3>=0) 
@variable(model,x4>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3*1
                -obs1[2,i]*obs1[4,i]*x2*x4*1
                -obs1[1,i]*obs1[4,i]*x1*1*x4
                -obs1[2,i]*obs1[3,i]*x2*1*x3
                -obs1[1,i]*obs1[2,i]*obs1[3,i]*obs1[4,i]*x1*x2*x3*x4)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2b5 (generic function with 3 methods)

In [309]:
function test2b11(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x2>=0) 
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x2<=1)
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*0
                -obs1[2,i]*obs1[4,i]*x2*x4*x5
                -obs1[1,i]*obs1[4,i]*1*x5*x4
                -obs1[2,i]*obs1[3,i]*0
                -obs1[1,i]*obs1[2,i]*obs1[3,i]*obs1[4,i]*x1*x2*0*x4)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2b11 (generic function with 3 methods)

In [352]:
function test2b12(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*0
                -obs1[2,i]*obs1[4,i]*x4*x5
                -obs1[1,i]*obs1[4,i]*x1*x5*x4
                -obs1[2,i]*obs1[3,i]*0
                -obs1[1,i]*obs1[2,i]*obs1[3,i]*obs1[4,i]*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2b12 (generic function with 3 methods)

In [311]:
function test2b13(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x4>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x4<=sqrt(k))
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*0
                -obs1[2,i]*obs1[4,i]*x2*x4
                -obs1[1,i]*obs1[4,i]*x1*x4
                -obs1[2,i]*obs1[3,i]*0
                -obs1[1,i]*obs1[2,i]*obs1[3,i]*obs1[4,i]*x1*x2*0*x4)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2b13 (generic function with 3 methods)

In [312]:
function test2b14(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x2>=0)
@variable(model,x3>=0) 
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x3*x5
                -obs1[2,i]*obs1[4,i]*0
                -obs1[1,i]*obs1[4,i]*0
                -obs1[2,i]*obs1[3,i]*x2*x5*x3
                -obs1[1,i]*obs1[2,i]*obs1[3,i]*obs1[4,i]*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2b14 (generic function with 3 methods)

In [313]:
function test2b15(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x3>=0) 
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3*x5
                -obs1[2,i]*obs1[4,i]*0*x5
                -obs1[1,i]*obs1[4,i]*x1*x5*0
                -obs1[2,i]*obs1[3,i]*x5*x3
                -obs1[1,i]*obs1[2,i]*obs1[3,i]*obs1[4,i]*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2b15 (generic function with 3 methods)

In [314]:
function test2b16(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x3>=0) 
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3
                -obs1[2,i]*obs1[4,i]*0
                -obs1[1,i]*obs1[4,i]*0
                -obs1[2,i]*obs1[3,i]*x2*x3
                -obs1[1,i]*obs1[2,i]*obs1[3,i]*obs1[4,i]*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2b16 (generic function with 3 methods)

In [315]:
function test2b17(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x2>=0)
@variable(model,x3>=0) 
@variable(model,x4>=0)
#set each one variable to be larger than 0
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@NLconstraint(model,x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x3
                -obs1[2,i]*obs1[4,i]*x2*x4
                -obs1[1,i]*obs1[4,i]*x4
                -obs1[2,i]*obs1[3,i]*x2*x3
                -obs1[1,i]*obs1[2,i]*obs1[3,i]*obs1[4,i]*x2*x3*x4)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2b17 (generic function with 3 methods)

In [316]:
function test2b18(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x2>=0)
@variable(model,x3>=0) 
@variable(model,x4>=0)
#set each one variable to be larger than 0
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@NLconstraint(model,x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3
                -obs1[2,i]*obs1[4,i]*x2*x4
                -obs1[1,i]*obs1[4,i]*x4
                -obs1[2,i]*obs1[3,i]*x2*x3
                -obs1[1,i]*obs1[2,i]*obs1[3,i]*obs1[4,i]*x2*x3*x4)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2b18 (generic function with 3 methods)

In [317]:
function test2b21(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x2>=0)
@variable(model,x4>=0)
#set each one variable to be larger than 0
@constraint(model,x2<=1)
@constraint(model,x4<=sqrt(k))
@NLconstraint(model,x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*0
                -obs1[2,i]*obs1[4,i]*x2*x4
                -obs1[1,i]*obs1[4,i]*x4
                -obs1[2,i]*obs1[3,i]*0
                -obs1[1,i]*obs1[2,i]*obs1[3,i]*obs1[4,i]*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2b21 (generic function with 3 methods)

In [318]:
function test2b22(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x4>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x4<=sqrt(k))
@NLconstraint(model,x1 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*0
                -obs1[2,i]*obs1[4,i]*x4
                -obs1[1,i]*obs1[4,i]*x1*x4
                -obs1[2,i]*obs1[3,i]*0
                -obs1[1,i]*obs1[2,i]*obs1[3,i]*obs1[4,i]*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2b22 (generic function with 3 methods)

In [319]:
function test2b23(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x2>=0)
@variable(model,x3>=0) 
#set each one variable to be larger than 0
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@NLconstraint(model,x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x3
                -obs1[2,i]*obs1[4,i]*0
                -obs1[1,i]*obs1[4,i]*0
                -obs1[2,i]*obs1[3,i]*x2*x3
                -obs1[1,i]*obs1[2,i]*obs1[3,i]*obs1[4,i]*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2b23 (generic function with 3 methods)

In [320]:
function test2b24(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x3>=0) 
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x3<=sqrt(k))
@NLconstraint(model,x1 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[3,i]*x1*x3
                -obs1[2,i]*obs1[4,i]*0
                -obs1[1,i]*obs1[4,i]*0
                -obs1[2,i]*obs1[3,i]*x3
                -obs1[1,i]*obs1[2,i]*obs1[3,i]*obs1[4,i]*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test2b24 (generic function with 3 methods)

In [321]:
function test3(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x3>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x2*x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[4,i]*x1*x4
                -obs1[2,i]*obs1[3,i]*x2*x3
                -obs1[1,i]*obs1[3,i]*x1*x3*x5
                -obs1[2,i]*obs1[4,i]*x2*x4*x5)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test3 (generic function with 3 methods)

In [322]:
function test31(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x2>=0)
@variable(model,x3>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x2*x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[4,i]*x4
                -obs1[2,i]*obs1[3,i]*x2*x3
                -obs1[1,i]*obs1[3,i]*x3*x5
                -obs1[2,i]*obs1[4,i]*x2*x4*x5)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test31 (generic function with 2 methods)

In [323]:
function test32(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x3>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[4,i]*x1*x4
                -obs1[2,i]*obs1[3,i]*x3
                -obs1[1,i]*obs1[3,i]*x1*x3*x5
                -obs1[2,i]*obs1[4,i]*x4*x5)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test32 (generic function with 2 methods)

In [324]:
function test33(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x2*x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[4,i]*x1*x4
                -obs1[2,i]*obs1[3,i]*0
                -obs1[1,i]*obs1[3,i]*x1*0*x5
                -obs1[2,i]*obs1[4,i]*x2*x4*x5)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test33 (generic function with 2 methods)

In [325]:
function test34(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x3>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x2*x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[4,i]*0
                -obs1[2,i]*obs1[3,i]*x2*x3
                -obs1[1,i]*obs1[3,i]*x1*x3*x5
                -obs1[2,i]*obs1[4,i]*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test34 (generic function with 2 methods)

In [326]:
function test35(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x3>=0)
@variable(model,x4>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[4,i]*x1*x4
                -obs1[2,i]*obs1[3,i]*x2*x3
                -obs1[1,i]*obs1[3,i]*x1*x3
                -obs1[2,i]*obs1[4,i]*x2*x4)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test35 (generic function with 2 methods)

In [327]:
function test311(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x3>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[4,i]*x4
                -obs1[2,i]*obs1[3,i]*x3
                -obs1[1,i]*obs1[3,i]*x3*x5
                -obs1[2,i]*obs1[4,i]*x4*x5)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test311 (generic function with 2 methods)

In [328]:
function test312(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x2>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x2<=1)
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x2*x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[4,i]*x4
                -obs1[2,i]*obs1[3,i]*0
                -obs1[1,i]*obs1[3,i]*0
                -obs1[2,i]*obs1[4,i]*x2*x4*x5)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test312 (generic function with 2 methods)

In [329]:
function test313(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x2>=0)
@variable(model,x3>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x2*x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[4,i]*0
                -obs1[2,i]*obs1[3,i]*x2*x3
                -obs1[1,i]*obs1[3,i]*x3*x5
                -obs1[2,i]*obs1[4,i]*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test313 (generic function with 2 methods)

In [330]:
function test314(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x2>=0)
@variable(model,x3>=0)
@variable(model,x4>=0)
#set each one variable to be larger than 0
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@NLconstraint(model,x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[4,i]*x4
                -obs1[2,i]*obs1[3,i]*x2*x3
                -obs1[1,i]*obs1[3,i]*x3
                -obs1[2,i]*obs1[4,i]*x2*x4)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test314 (generic function with 2 methods)

In [331]:
function test315(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x4>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x4<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[4,i]*x1*x4
                -obs1[2,i]*obs1[3,i]*0
                -obs1[1,i]*obs1[3,i]*0
                -obs1[2,i]*obs1[4,i]*x4*x5)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test315 (generic function with 2 methods)

In [332]:
function test316(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x3>=0)
@variable(model,x5>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x5<=1)
@NLconstraint(model,x1*x5 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[4,i]*0
                -obs1[2,i]*obs1[3,i]*x3
                -obs1[1,i]*obs1[3,i]*x1*x3*x5
                -obs1[2,i]*obs1[4,i]*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test316 (generic function with 2 methods)

In [333]:
function test317(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x3>=0)
@variable(model,x4>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x3<=sqrt(k))
@constraint(model,x4<=sqrt(k))
@NLconstraint(model,x1 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[4,i]*x1*x4
                -obs1[2,i]*obs1[3,i]*x3
                -obs1[1,i]*obs1[3,i]*x1*x3
                -obs1[2,i]*obs1[4,i]*x4)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test317 (generic function with 2 methods)

In [334]:
function test318(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x4>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x4<=sqrt(k))
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[4,i]*x1*x4
                -obs1[2,i]*obs1[3,i]*0
                -obs1[1,i]*obs1[3,i]*0
                -obs1[2,i]*obs1[4,i]*x2*x4)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test318 (generic function with 2 methods)

In [335]:
function test319(freqtest,e)
model = Model(Ipopt.Optimizer)
@variable(model,x1>=0)
@variable(model,x2>=0)
@variable(model,x3>=0)
#set each one variable to be larger than 0
@constraint(model,x1<=1)
@constraint(model,x2<=1)
@constraint(model,x3<=sqrt(k))
@NLconstraint(model,x1*x2 == e)
@NLobjective(model, Min, 1/2*sum(((freqtest[i]-obs1[1,i]*obs1[4,i]*0
                -obs1[2,i]*obs1[3,i]*x2*x3
                -obs1[1,i]*obs1[3,i]*x1*x3
                -obs1[2,i]*obs1[4,i]*0)^2)/q[i] for i in 1:16))
optimize!(model)
status = termination_status(model)
asdgf =objective_value(model)
    return objective_value(model)
end

test319 (generic function with 2 methods)

In [337]:
function getforonetime(freqtest,e)
    testwhole = zeros(4,25)
    testwhole = settest1(testwhole,freqtest,e)
    testwhole = settest2(testwhole,freqtest,e)
    testwhole = settest3(testwhole,freqtest,e)
    testwhole = settest4(testwhole,freqtest,e)
    for i in 1:4
        for j in 1:25
            if testwhole[i,j] == 0
                testwhole[i,j] = 100
            end
        end
    end
    return testwhole
end

getforonetime (generic function with 2 methods)

In [336]:
function settest1(testwhole,freqtest,e)
    testwhole[1,1] = test1(freqtest,e)
    testwhole[1,2] = test11(freqtest,e)
    testwhole[1,3] = test12(freqtest,e)
    testwhole[1,4] = test13(freqtest,e)
    testwhole[1,5] = test14(freqtest,e)
    testwhole[1,6] = test15(freqtest,e)
    testwhole[1,7] = test111(freqtest,e)
    testwhole[1,8] = test112(freqtest,e)
    testwhole[1,9] = test113(freqtest,e)
    testwhole[1,10] = test114(freqtest,e)
    testwhole[1,11] = test115(freqtest,e)
    testwhole[1,12] = test116(freqtest,e)
    testwhole[1,13] = test117(freqtest,e)
    testwhole[1,14] = test118(freqtest,e)
    testwhole[1,15] = test1111(freqtest,e)
    testwhole[1,16] = test1112(freqtest,e)
    testwhole[1,17] = test1113(freqtest,e)
    testwhole[1,18] = test1114(freqtest,e)
    testwhole[1,19] = test1115(freqtest,e)
    testwhole[1,20] = test1116(freqtest,e)
    return testwhole
end

settest1 (generic function with 3 methods)

In [338]:
function settest2(testwhole,freqtest,e)
    testwhole[2,1] = test2a(freqtest,e)
    testwhole[2,2] = test2a1(freqtest,e)
    testwhole[2,3] = test2a2(freqtest,e)
    testwhole[2,4] = test2a3(freqtest,e)
    testwhole[2,5] = test2a4(freqtest,e)
    testwhole[2,6] = test2a5(freqtest,e)
    testwhole[2,7] = test2a11(freqtest,e)
    testwhole[2,8] = test2a12(freqtest,e)
    testwhole[2,9] = test2a13(freqtest,e)
    testwhole[2,10] = test2a14(freqtest,e)
    testwhole[2,11] = test2a15(freqtest,e)
    testwhole[2,12] = test2a16(freqtest,e)
    testwhole[2,13] = test2a17(freqtest,e)
    testwhole[2,14] = test2a18(freqtest,e)
    return testwhole
end

settest2 (generic function with 3 methods)

In [339]:
function settest3(testwhole,freqtest,e)
    testwhole[3,1] = test2b(freqtest,e)
    testwhole[3,2] = test2b1(freqtest,e)
    testwhole[3,3] = test2b2(freqtest,e)
    testwhole[3,4] = test2b3(freqtest,e)
    testwhole[3,5] = test2b4(freqtest,e)
    testwhole[3,6] = test2b11(freqtest,e)
    testwhole[3,7] = test2b12(freqtest,e)
    testwhole[3,8] = test2b13(freqtest,e)
    testwhole[3,9] = test2b14(freqtest,e)
    testwhole[3,10] = test2b15(freqtest,e)
    testwhole[3,11] = test2b16(freqtest,e)
    testwhole[3,12] = test2b17(freqtest,e)
    testwhole[3,13] = test2b18(freqtest,e)
    testwhole[3,14] = test2b21(freqtest,e)
    testwhole[3,16] = test2b22(freqtest,e)
    testwhole[3,17] = test2b23(freqtest,e)
    testwhole[3,18] = test2b24(freqtest,e)
    return testwhole
end

settest3 (generic function with 3 methods)

In [348]:
function settest4(testwhole,freqtest,e)
    testwhole[4,1] = test3(freqtest,e)
    testwhole[4,2] = test31(freqtest,e)
    testwhole[4,3] = test32(freqtest,e)
    testwhole[4,4] = test33(freqtest,e)
    testwhole[4,5] = test34(freqtest,e)
    testwhole[4,6] = test35(freqtest,e)
    testwhole[4,7] = test311(freqtest,e)
    testwhole[4,8] = test312(freqtest,e)
    testwhole[4,9] = test313(freqtest,e)
    testwhole[4,10] = test314(freqtest,e)
    testwhole[4,11] = test315(freqtest,e)
    testwhole[4,12] = test316(freqtest,e)
    testwhole[4,13] = test317(freqtest,e)
    testwhole[4,14] = test318(freqtest,e)
    return testwhole
end

settest4 (generic function with 3 methods)

In [340]:
count = zeros(4,25)

4×25 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [344]:
function setcount(e)
    test123 = rand(d)
    freqtest = (test123-mean)/sqrt(k)
    testwhole = getforonetime(freqtest,e)
    sat = findmin(testwhole)
    count[sat[2]] = count[sat[2]]+1
    return testwhole
end

setcount (generic function with 2 methods)

In [360]:
for i in 1:10
    setcount(0.5)
end

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        3
Number of nonzeros in inequality constraint Jacobian.:        5
Number of nonzeros in Lagrangian Hessian.............:       21

Total number of variables............................:        5
                     variables with only lower bounds:        5
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        5
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        5

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  3.1029731e+00 5.00e-01 5.27e-02  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

In [361]:
count

4×25 Matrix{Float64}:
 30.0   2.0   9.0  0.0  0.0  1.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 10.0   7.0  70.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 81.0  49.0  30.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 29.0   4.0  10.0  0.0  0.0  0.0  2.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0